# Implementation of paper Bevilacqua et al.

## Libraries Prequisities

In [ ]:
!pip install scikit-learn
!pip install numpy==1.16.1

In [ ]:
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics.classification import accuracy_score, recall_score, f1_score
import scipy.stats as st

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
from tensorflow.keras.callbacks import Callback

In [ ]:
np.random.seed(12227)

In [ ]:
n_ep = 200
loss = 0.2
bs = 1000

In [ ]:
def custom_stopping(value=0.5, verbose=0):
    early = EarlyStoppingByLossVal(monitor='val_loss', value=value, verbose=verbose)
    return early

class EarlyStoppingByLossVal(Callback):
    def __init__(self, monitor='val_acc', value=0.95, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        # if current is None:
        # warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current < self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping THR" % epoch)
            self.model.stop_training = True

In [ ]:
def DataPreparation(data_input_file):
  print('Bevilacqua  et al. 2015 {}'.format(data_input_file))
  data = np.load(data_input_file)
  X = data['X']
  # X = X[:, 0, :, :]
  Y = data['y']
  folds = data['folds']
  # classes_number = Y.shape[1]
  # Y = np.argmax(Y, axis=1)
  return X,Y,folds

In [ ]:
X,Y,f=DataPreparation('/content/drive/My Drive/Computer_vision_project/data/LOSO/MHEALTH.npz')

In [ ]:
print(X[2].shape)

In [ ]:
def ReportAccuracies(avg_acc, avg_recall,avg_f1):
  ic_acc = st.t.interval(0.9, len(avg_acc) - 1, loc=np.mean(avg_acc), scale=st.sem(avg_acc))
  ic_recall = st.t.interval(0.9, len(avg_recall) - 1, loc=np.mean(avg_recall), scale=st.sem(avg_recall))
  ic_f1 = st.t.interval(0.9, len(avg_f1) - 1, loc=np.mean(avg_f1), scale=st.sem(avg_f1))
  print('Mean Accuracy[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_acc), ic_acc[0], ic_acc[1]))
  print('Mean Recall[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_recall), ic_recall[0], ic_recall[1]))
  print('Mean F1[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_f1), ic_f1[0], ic_f1[1]))

In [ ]:
def RunBevilacqua(data_input_file):
  X,Y,folds,= DataPreparation(data_input_file)
  avg_acc, avg_recall,avg_f1= Train(X,Y,folds)
  ReportAccuracies(avg_acc, avg_recall,avg_f1 )

In [ ]:
def build_model_1(row,col,num_classes):
  layers = [
        tf.keras.layers.Conv2D(filters=10,kernel_size=(3, 5), activation = tf.nn.relu),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPool2D(pool_size=(3,3),strides = 1),

        tf.keras.layers.Conv2D(filters=2,kernel_size=(2, 4),activation = tf.nn.relu),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPool2D(pool_size=(2,2),strides = 1),

        tf.keras.layers.Conv2D(filters=2,kernel_size=(2, 2),activation = tf.nn.relu),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPool2D(pool_size=(3,2),strides = 1),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(500, activation = tf.nn.relu),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(25, activation = tf.nn.relu),
        tf.keras.layers.Dropout(0.5),
       
        tf.keras.layers.Dense(num_classes, activation = 'softmax')
    ]
    
  model = tf.keras.Sequential(layers)
  return model

In [ ]:
def Train(X,y,folds):
  avg_acc = []
  avg_recall = []
  avg_f1 = []
  n_class = y.shape[1]
  _, _,img_rows, img_cols = X.shape
  print(X.shape)
  for i in range(0, len(folds)):
    train_idx = folds[i][0]
    test_idx = folds[i][1]
    X_train = X[train_idx]
    X_test = X[test_idx]
    model=build_model_1(img_rows,img_cols,n_class)
    # print(model.summary)
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='Adam')
    model.fit(X_train, y[train_idx], batch_size=64, epochs=300 )
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y[test_idx], axis=1)
    acc_fold = accuracy_score(y_true, y_pred)
    avg_acc.append(acc_fold)
    recall_fold = recall_score(y_true, y_pred, average='macro')
    avg_recall.append(recall_fold)
    f1_fold = f1_score(y_true, y_pred, average='macro')
    avg_f1.append(f1_fold)
    print('Accuracy[{:.4f}] Recall[{:.4f}] F1[{:.4f}] at fold[{}]'.format(acc_fold, recall_fold, f1_fold ,i+1))
    print('________________________________________________________________')
    
  return avg_acc, avg_recall,avg_f1

In [ ]:
tf.keras.backend.set_image_data_format('channels_first')

In [ ]:
RunBevilacqua('/content/drive/My Drive/Computer_vision_project/data/LOSO/MHEALTH.npz')

In [ ]:
RunBevilacqua('/content/drive/My Drive/Computer_vision_project/data/FNOW/MHEALTH.npz')

In [ ]:
RunBevilacqua('/content/drive/My Drive/Computer_vision_project/data/LOTO/MHEALTH.npz')

In [ ]:
RunBevilacqua('/content/drive/My Drive/Computer_vision_project/data/SNOW/MHEALTH.npz')